In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv('../.env')

True

# Chains & Agents

## Chains

### Simple Chains

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [14]:
template = 'Who won the Men\'s International Cricket World Cup in the year {year} & who was the captain ?'

In [15]:
prompt= PromptTemplate(
    input_variables=['year'],
    template=template
)

In [16]:
llm = OpenAI()

In [17]:
chain = LLMChain(llm=llm, prompt=prompt)

In [18]:
print(chain.run('1983'))



The Indian cricket team, led by Kapil Dev, won the 1983 Men's International Cricket World Cup.


### Simple Sequential Chains

In [19]:
from langchain.chains import SimpleSequentialChain
from langchain import HuggingFaceHub

In [25]:
template_1 = 'Who won the Men\'s International Cricket World Cup in the year {year} ?'
template_2 = 'who was the captain ? when {team}'

In [26]:
prompt_1 = PromptTemplate(
    input_variables=['year'],
    template=template_1
)

prompt_2 = PromptTemplate(
    input_variables=['team'],
    template=template_2
)

In [46]:
open_ai_llm = OpenAI()
open_ai_chain = LLMChain(llm=open_ai_llm, prompt=prompt_1)

# hf_llm = HuggingFaceHub(repo_id='meta-llama/Llama-2-7b')
hf_llm = OpenAI()
hf_chain = LLMChain(llm=hf_llm, prompt=prompt_2)

In [47]:
seq_chain = SimpleSequentialChain(
    chains=[open_ai_chain, hf_chain],
    verbose=True
)

In [48]:
seq_chain.run('1983')



> Entering new SimpleSequentialChain chain...


The Indian cricket team won the Men's International Cricket World Cup in 1983.
 The captain of the Indian team was Kapil Dev.

> Finished chain.


' The captain of the Indian team was Kapil Dev.'

### Utility Chains

#### Summarize chains

In [71]:
import inspect

from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [50]:
with open('./data/Tenerife_airport_disaster.txt') as f:
    data = f.read()

In [51]:
data[: 100]

'Tenerife airport disaster\n\nThe Tenerife airport disaster occurred on March 27, 1977, when two Boeing'

#### Split Text

In [64]:
txt_splitter = CharacterTextSplitter(
    chunk_size = 4100,
    chunk_overlap  = 500,
)
# Create data to Docs
docs = txt_splitter.create_documents([data])

# Alternate method to create docs
# txt = txt_splitter.split_text(data)
# docs = [Document(page_content=t) for t in txt]

docs

[Document(page_content="Tenerife airport disaster\n\nThe Tenerife airport disaster occurred on March 27, 1977, when two Boeing 747 passenger jets collided on the runway at Los Rodeos Airport (now Tenerife North Airport) on the Spanish island of Tenerife. The collision occurred when KLM Flight 4805 initiated its takeoff run during dense fog while Pan Am Flight 1736 was still on the runway. The impact and resulting fire killed everyone on board KLM 4805 and most of the occupants of Pan Am 1736, with only 61 survivors in the front section of the aircraft. With 583 fatalities, the disaster is the deadliest accident in aviation history.\nA bomb set off by the Canary Islands Independence Movement at Gran Canaria Airport had caused many flights to be diverted to Los Rodeos, including the two aircraft involved in the accident. The airport quickly became congested with parked airplanes blocking the only taxiway and forcing departing aircraft to taxi on the runway instead. Patches of thick fog w

In [65]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=True
)

In [67]:
print(chain.run(docs))



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Tenerife airport disaster

The Tenerife airport disaster occurred on March 27, 1977, when two Boeing 747 passenger jets collided on the runway at Los Rodeos Airport (now Tenerife North Airport) on the Spanish island of Tenerife. The collision occurred when KLM Flight 4805 initiated its takeoff run during dense fog while Pan Am Flight 1736 was still on the runway. The impact and resulting fire killed everyone on board KLM 4805 and most of the occupants of Pan Am 1736, with only 61 survivors in the front section of the aircraft. With 583 fatalities, the disaster is the deadliest accident in aviation history.
A bomb set off by the Canary Islands Independence Movement at Gran Canaria Airport had caused many flights to be diverted to Los Rodeos, including the two aircraft involved in the accident. The airport quickly became congested with

#### LLMRequestsChain

In [68]:
from langchain.chains import LLMRequestsChain

In [69]:
template = '''
Extract the answer from the question '{query}'. Say no results found if there's not info
{requests_result}
'''

prompt = PromptTemplate(
    input_variables=['query', 'requests_result'],
    template=template
)

llm = OpenAI()

chain = LLMRequestsChain(
    llm_chain=LLMChain(llm=llm, prompt=prompt)
)

query = 'Who won the 1983 Cricket world up ?'

input_query = {
    'query': query,
    'url': f'https://www.google.com/search?q={query.replace(" ", "+")}'
}

chain(inputs=input_query)

{'query': 'Who won the 1983 Cricket world up ?',
 'url': 'https://www.google.com/search?q=Who+won+the+1983+Cricket+world+up+?',
 'output': '\nAnswer: India national cricket team'}

##### Inspect the Chain Function

In [73]:
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}

